In [6]:
# import libraries

import openai
import os
from dotenv import load_dotenv, find_dotenv
from langchain.evaluation import load_evaluator
from langchain.llms import OpenAI
import gradio as gr

import pandas as pd
import json

In [7]:
import warnings
warnings.filterwarnings("ignore")

In [8]:
# load and read data.json

with open('data.json') as f:
  data = json.load(f)

print(data)

{'argumentative': [{'title': 'Careful dieting is the most important part of a healthy life. Discuss.', 'good-example': 'Dieting is often associated with weight loss and restriction, but it is also about making conscious and informed choices about the foods we eat. A healthy diet is a crucial aspect of overall health and well-being, as the foods we consume provide the necessary nutrients and energy to fuel our bodies. One of the main reasons why careful dieting is so important is that it helps to prevent over-eating and the associated health risks. Consuming too many calories can lead to weight gain, which in turn increases the risk of chronic diseases such as diabetes, heart disease and certain cancers. By paying attention to portion sizes and making informed choices about the foods we eat, we can help to control our calorie intake and maintain a healthy weight. In addition to controlling calorie intake, careful dieting also involves avoiding certain foods that can be harmful to our he

In [17]:
# load the env.txt that contains the OPENAI key

load_dotenv('env.txt')
openai_api_key = os.getenv('OPENAI_API_KEY')

In [ ]:
load_dotenv('essay.txt')
textType = None #Matthias please put the input of textType from Gradio Here

In [19]:
llm = OpenAI(
    openai_api_key=openai_api_key,
    model ='gpt-3.5-turbo-instruct'
)

In [32]:
narrative_custom_criterion = {
    "grammar": (
        "Does the essay follow proper grammar rules?"
        "Does the essay follow proper pronunciation rules?"
        "Does the output use a robust dictionary of vocabulary?"
    ),
    "coherence": "Does the output have a coherent flow of ideas?",
    "relevance":"Does the output fully address the task?",
    "description":"Does the output include sensory words?",
    "characters":"Does the output capture the personality or emotions of the characters in the story?",
    "openings": "Does the output have an engaging opening?"
}

In [33]:
argumentative_custom_criterion = {
    "grammar": "Does the output follow proper grammar, spelling, and pronunciation rules?",
    "coherence": "Does the output have a coherent flow of ideas?",
    "vocabulary":"Does the output use a robust dictionary of vocabulary?",
    "relevance":"Does the output fully address the task?",
    "attention":"Does the output capture your attention?",
    
}

In [23]:
if textType == "Narrative":
        evaluator = load_evaluator("pairwise_string", criteria="custom_criterion", llm=llm)

This chain was only tested with GPT-4. Performance may be significantly worse with other models.


In [24]:
print(evaluator)

prompt=ChatPromptTemplate(input_variables=['input', 'prediction', 'prediction_b'], partial_variables={'reference': '', 'criteria': 'For this evaluation, you should primarily consider the following criteria:\ncustom_criterion'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='Please act as an impartial judge and evaluate the quality of the responses provided by two AI assistants to the user question displayed below. You should choose the assistant that follows the user\'s instructions and answers \the user\'s question better. Your evaluation should consider factors such as the helpfulness, relevance, accuracy, depth, creativity, and level of detail of their responses. Begin your evaluation by comparing the two responses and provide a short explanation. Avoid any position biases and ensure that the order in which the responses were presented does not influence your decision. Do not allow the length of the responses to influence your evaluation. D

In [25]:
def return_feedback():
  return ""

In [26]:
def predict(message, history):

    output = str(return_feedback(message)).replace("\n", "<br/>")
    return output

In [27]:
demo = gr.ChatInterface(
    predict,
    title = f'Olevel English Essay Grader'
)